## Neste notebook, você codificará do zero seu primeiro agente de Reinforcement Learning jogando FrozenLake ❄️ usando Q-Learning


Adaptado HuggingFace

<img src="https://www.gymlibrary.dev/_images/frozen_lake.gif" alt="Environments"/>

###🎮 Environments:

>

- [FrozenLake-v1](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/)


###📚 RL-Library:

- Python and NumPy
- [Gym](https://www.gymlibrary.dev/)

## Pequena revisão de Q-Learning

- O *Q-Learning* **é o algoritmo RL que**

   - Treina *Q-Function*, uma **função ação-valor (action-value function)** que contém, como memória interna, uma *Q-table* **que contém todos os valores do par estado-ação.**

   - Dado um estado e uma ação, nossa Q-Function **pesquisará em sua Q-table o valor correspondente.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-function-2.jpg" alt="Q function"  width="100%"/>

- Quando o treinamento é concluído,**temos uma Função-Q ideal, portanto, uma Tabela-Q ideal.**

- E se **tivermos uma função Q ótima**,
ter uma política ideal, pois **sabemos para cada estado qual é a melhor ação a ser tomada.**

Mas, no começo, nossa **Q-Table é inútil, pois fornece um valor arbitrário para cada par estado-ação (na maioria das vezes, inicializamos a Q-Table com valores 0)**. Mas, conforme vamos explorando o ambiente e atualizando nosso Q-Table, ele nos dará aproximações cada vez melhores

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/q-learning.jpeg" alt="q-learning.jpeg" width="100%"/>

This is the Q-Learning pseudocode:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg" alt="Q-Learning" width="100%"/>


# Let's code our first Reinforcement Learning algorithm 🚀

In [1]:
import gymnasium as gym

# O modo 'human' abre uma janela nativa no Windows
env = gym.make("CartPole-v1", render_mode="human")
env.reset()

(array([ 0.02533983,  0.04777611,  0.00611492, -0.00021378], dtype=float32),
 {})

In [34]:
import numpy as np
import gymnasium as gym
import random
import os
import tqdm
import imageio

import pickle as pickle
# from tqdm.notebook import tqdm
from tqdm import tqdm

In [3]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False)

In [4]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Discrete(16)
Sample observation 2


In [10]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 3


In [11]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  16  possible states
There are  4  possible actions


In [12]:
# Vamos criar nossa Qtable de tamanho (state_space, action_space) e inicializar cada valor em 0 usando np.zeros
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [22]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

Qtable_frozenlake

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

## Defina a política gananciosa 🤖
Lembre-se de que temos duas políticas, pois o Q-Learning é um algoritmo **off-policy**. Isso significa que estamos usando uma **política diferente para atuar e atualizar a função de valor**.

- Política Epsilon-gananciosa (política de atuação)
- Greedy-policy (política de atualização)

A política gananciosa também será a política final que teremos quando o agente Q-learning for treinado. A política gulosa é usada para selecionar uma ação da tabela Q.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-4.jpg" alt="Q-Learning" width="100%"/>


In [15]:
def greedy_policy(Qtable, state):
  # Exploitation
  action = np.argmax(Qtable[state][:])

  return action

def epsilon_greedy_policy(Qtable, state, epsilon):
  # Gera aleatoriamente um número entre 0 e 1
  random_int = random.uniform(0,1)
  # if random_int > maior que epsilon --> exploitation
  if random_int > epsilon:
     # Execute a ação com o maior valor dado um estado
     # np.argmax pode ser útil aqui
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

## Definindo os hiperparâmetros ⚙️
Os hiperparâmetros relacionados à exploração são alguns dos mais importantes.

- Precisamos garantir que nosso agente **explore o espaço de estados** o suficiente para aprender uma boa aproximação de valor. Para fazer isso, precisamos ter decaimento progressivo do epsilon.
- Se você diminuir o epsilon muito rápido (decay_rate muito alto), **você corre o risco de que seu agente fique preso**, já que seu agente não explorou o espaço de estado o suficiente e, portanto, não pode resolver o problema.

In [21]:
# Parâmetros de treinamento
n_training_episodes = 10000 # Total de episódios de treinamento
learning_rate = 0.7 # Taxa de aprendizado

# Parâmetros de avaliação
n_eval_episodes = 100 # Número total de episódios de teste

# Parâmetros do ambiente
env_id = "FrozenLake-v1" # Nome do ambiente
max_steps = 99 # Max passos por episódio
gamma = 0.95 # Taxa de desconto
eval_seed = [] # A semente de avaliação do ambiente

# Parâmetros de exploração
max_epsilon = 1.0 # Probabilidade de exploração no início
min_epsilon = 0.05 # Probabilidade mínima de exploração
decay_rate = 0.0005 # Taxa de decaimento exponencial para prob de exploração

## Rotina de Treinamento

O loop de treinamento é assim:
```
Por episódio no total de episódios de treino:

Reduza o epsilon (já que precisamos cada vez menos de exploração)
Redefinir o ambiente

   Para passo em passos de tempo máximo:
     Escolha a ação At usar a política gananciosa do epsilon
     Tome a ação (a) e observe o(s) estado(s) resultante(s) e a recompensa (r)
     Atualize o valor Q Q(s,a) usando a equação de Bellman Q(s,a) + lr [R(s,a) + gama * max Q(s',a') - Q(s,a)]
     Se terminar, termine o episódio
     Nosso próximo estado é o novo estado
```

In [26]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    # Reduzir epsilon (exploration vs exploitation)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

    # 1. AJUSTE AQUI: O reset no Gymnasium retorna 'state' e 'info'
    state, info = env.reset()

    # repete para cada passo dentro do episódio
    for step in range(max_steps):
      # Escolha a ação usando a epsilon greedy policy
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # 2. Executa a ação no ambiente (já estava certo com 5 variáveis)
      new_state, reward, terminated, truncated, info = env.step(action)

      # 3. AJUSTE AQUI: Atualiza a variável 'done' combinando terminated e truncated
      done = terminated or truncated

      # Update Q(s,a) := Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      # (Garantindo que learning_rate e gamma estejam definidos globalmente no seu notebook)
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # Verifica se o episódio acabou (caiu no buraco, pegou o frisbee ou estourou o limite de tempo)
      if done:
        break

      # Atualiza o estado atual para o próximo passo
      state = new_state

  return Qtable

## Treinando o agente Q-Learning 🏃

In [27]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)


100%|██████████| 10000/10000 [00:00<00:00, 10256.99it/s][A


In [28]:
Qtable_frozenlake

array([[0.73509189, 0.77378094, 0.77378094, 0.73509189],
       [0.73509189, 0.        , 0.81450625, 0.77378094],
       [0.77378094, 0.857375  , 0.77378094, 0.81450625],
       [0.81450625, 0.        , 0.77378094, 0.77378094],
       [0.77378094, 0.81450625, 0.        , 0.73509189],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.        , 0.81450625],
       [0.        , 0.        , 0.        , 0.        ],
       [0.81450625, 0.        , 0.857375  , 0.77378094],
       [0.81450625, 0.9025    , 0.9025    , 0.        ],
       [0.857375  , 0.95      , 0.        , 0.857375  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.95      , 0.857375  ],
       [0.9025    , 0.95      , 1.        , 0.9025    ],
       [0.        , 0.        , 0.        , 0.        ]])

## Avaliação do Método 📝

- Definimos o método de avaliação que vamos usar para testar nosso agente Q-Learning.

In [31]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
   Avalie o agente para episódios ``n_eval_episodes`` e retorne recompensa média e desvio padrão de recompensa.
   :param env: O ambiente de avaliação
   :param n_eval_episodes: Número de episódios para avaliar o agente
   :param Q: A tabela Q
   :param seed: A matriz de sementes de avaliação (para taxi-v3)
   """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    # 1. AJUSTE AQUI: Desempacotar 'state' e 'info' do reset, com ou sem seed
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()

    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
      action = greedy_policy(Q, state)

      # 2. AJUSTE AQUI: Receber as 5 variáveis do Gymnasium
      new_state, reward, terminated, truncated, info = env.step(action)

      # 3. AJUSTE AQUI: Atualizar o status de 'done'
      done = terminated or truncated

      total_rewards_ep += reward

      if done:
        break

      state = new_state

    episode_rewards.append(total_rewards_ep)

  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Avaliando nosso agente Q-Learning 📈

- Normalmente, você deve ter uma recompensa média de 1,0
- O **ambiente é relativamente fácil** já que o espaço de estados é muito pequeno (16). O que você pode tentar fazer é [substituí-lo pela versão escorregadia](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/), que introduz estocasticidade, tornando o ambiente mais complexo.

In [32]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")


100%|██████████| 100/100 [00:00<00:00, 29595.71it/s]

Mean_reward=1.00 +/- 0.00


In [39]:
def record_video(env, Qtable, out_directory, fps=1):
  """
   Gerar um vídeo de replay do agente
   :param env: O ambiente criado com render_mode='rgb_array'
   :param Qtable: Qtable do nosso agente
   :param out_directory: Caminho para salvar o vídeo (ex: 'replay.mp4')
   :param fps: quantos quadros por segundo (com taxi-v3 e frozenlake-v1 usamos 1)
   """
  images = []
  done = False

  # 1. AJUSTE: Desempacotar state e info no reset
  state, info = env.reset(seed=random.randint(0,500))

  # 2. AJUSTE: O Gymnasium moderno não usa mais mode='' aqui.
  img = env.render()
  images.append(img)

  while not done:
    # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
    action = np.argmax(Qtable[state][:])

    # 3. AJUSTE: Receber as 5 variáveis do step
    state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    img = env.render()
    images.append(img)

  # Salvar o vídeo
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [40]:
# 1. Crie um NOVO ambiente com o render_mode="rgb_array"
# (Atenção: ajuste o map_name e is_slippery para ficarem iguais ao do seu treino)
env_video = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode="rgb_array")

# 2. Defina o nome do arquivo
video_path = "replay.mp4"

# 3. Grave o vídeo passando o novo ambiente (env_video)
# Dica: Use fps=1. Valores quebrados como 0.5 podem dar erro em alguns geradores de vídeo do Windows.
record_video(env_video, Qtable_frozenlake, video_path, fps=1)

# 4. Feche o ambiente após gravar para liberar a memória
env_video.close()

print("Vídeo gravado com sucesso!")

Vídeo gravado com sucesso!
